# ${Rabin}$ ${Karp}$算法

# 算法介绍

> Rabin Karp 算法：简称为 RK 算法。是由它的两位发明者 Michael Oser Rabin 和 Richard Manning Karp 的名字来命名的。RK 算法是他们在 1987 年提出的、使用哈希函数以在文本中搜寻单个模式串的字符串搜索算法。
> - Rabin Karp 算法思想：对于给定文本串 $T$ 与模式串 $p$，通过滚动哈希算快速筛选出与模式串 $p$ 不匹配的文本位置，然后在其余位置继续检查匹配项。

# 算法步骤

## 整体步骤

1. 对于给定的文本串与模式串，求出文本串的长度为n，模式串的长度为m。
2. 通过滚动哈希算法求出模式串的哈希值$hash\_p$。
3. 再通过滚动哈希算法对文本串中$n-m+1$个子串分别求哈希值$hash\_t$。
4. 然后逐个与模式串的哈希值比较大小。
    1. 如果当前子串的哈希值$hash\_t$与模式串哈希值$hash\_p$不同，则说明两者不匹配，则继续向后匹配。
    2. 如果当前子串的哈希值$hash\_t$与模式串的哈希值$hash\_p$相等，则验证当前子串和模式串的每个字符是否真的相等（避免哈希冲突）。
        1. 如果当前子串和模式串的每个字符相等，则说明当前子串和模式串匹配。
        2. 如果当前子串和模式串的每个字符不相等，则说明两者不匹配，继续向后匹配。
5. 比较到末尾，如果仍未成功匹配，则说明文本串中不包含模式串，方法返回-1。

## 滚动哈希算法

给定的文本串与模式串，求出文本串的长度为n，模式串的长度为m。字符串字符种类数为d，则：
- 模式串的哈希值计算方式为：$Hash(p) = p_0 \times d^{m - 1} + p_1 \times d^{m - 2} + … + p_{m-1} \times d^{0}$。
- 文本串中起始于位置0，长度为m的子串$T_{[0, m - 1]}$对应哈希值计算方法为：$Hash(T_{[0, m - 1]}) = T_0 \times d^{m - 1} + T_1 \times d^{m - 2} + ... + T_{m - 1} \times d^0$。
- 已知子串的哈希值$Hash(T_{[i,i+m-1]})$，将子串向右移动一位的子串对应哈希值计算方法为：$Hash(T_{[i + 1, i + m]}) = [Hash(T_{[i, i + m - 1]}) - T_i \times d^{m - 1}] \times d + T_{i + m} \times d^{0}$。

因为哈希值过大会造成溢出，所以我们在计算过程中还要对结果取模。取模的值应该尽可能大，并且应该是质数，这样才能减少哈希碰撞的概率。

# 算法代码实现

In [ ]:
# T 为文本串，p 为模式串，d 为字符集的字符种类数，q 为质数
def rabinKarp(T: str, p: str, d, q) -> int:
    n, m = len(T), len(p)
    if n < m:
        return -1
    
    hash_p, hash_t = 0, 0
    
    for i in range(m):
        hash_p = (hash_p * d + ord(p[i])) % q           # 计算模式串 p 的哈希值
        hash_t = (hash_t * d + ord(T[i])) % q           # 计算文本串 T 中第一个子串的哈希值
    
    power = pow(d, m - 1) % q                           # power 用于移除字符哈希时
    
    for i in range(n - m + 1):
        if hash_p == hash_t:                            # 检查模式串 p 的哈希值和子串的哈希值
            match = True                                # 如果哈希值相等，验证模式串和子串每个字符是否完全相同（避免哈希冲突）
            for j in range(m):
                if T[i + j] != p[j]:
                    match = False                       # 模式串和子串某个字符不相等，验证失败，跳出循环
                    break
            if match:                                   # 如果模式串和子串每个字符是否完全相同，返回匹配开始位置
                return i
        if i < n - m:                                   # 计算下一个相邻子串的哈希值
            hash_t = (hash_t - power * ord(T[i])) % q   # 移除字符 T[i]
            hash_t = (hash_t * d + ord(T[i + m])) % q   # 增加字符 T[i + m]
            hash_t = (hash_t + q) % q                   # 确保 hash_t >= 0
        
    return -1

# 算法分析

RK 算法可以看做是 BF 算法的一种改进。在 BF 算法中，每一个字符都需要进行比较。而在 RK 算法中，判断模式串的哈希值与每个子串的哈希值之间是否相等的时间复杂度为$O(1)$。总共需要比较$n-m+1$个子串的哈希值，所以 RK 算法的整体时间复杂度为$O(n)$。跟 BF 算法相比，RK 算法的效率提高了很多。

但是如果存在冲突的情况下，算法的效率会降低。最坏情况是每一次比较模式串的哈希值和子串的哈希值时都相等，但是每一次都会出现冲突，那么每一次都需要验证模式串和子串每个字符是否完全相同，那么总的比较次数就是$m×(n-m+1)$，时间复杂度就会退化为$O(m×n)$。